# Study Setup & Configuration

I think this notebook needs to take care of explaining how a study should be setup. In particularly, we need to outline (1) how to use the configuration files and (2) what data is needed to make this run and what that data needs to look at. 

## Intro to Configuration Files
Configuration options are organized in YAML files - text files that contain key: value pairs of strings, numbers, and lists, where keys can be nested via indentation, to organize information hierarchically. We use YAML config files to specify the settings used when running range_driver.

## Fields
### Required Fields
### Optional Fields

## Using Multiple Configuration Files
To separate the definitions of default values and study specific settings, configurations can be loaded in layers, where a later file can update settings from an earlier file. It is also possible to merge all information into a single config file, if that is preferred.

YAML configuration in layers: system defaults, view settings (merge with system?), study specific

## Choosing Study Boundaries
This will need to be updated with Steven's work on multiple boundaries for different data sources. 

Specific Qs I would have as a user: 
* Why are there so many configuration files? How do they get used together?
* How do I read the configuration files? 
* Can I add anything to the config file? Relatedly -- what config options exist? 

